In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)

In [2]:
current_date = dt.date.today()
day = current_date.day
month = current_date.month
year = current_date.year

way = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Илларионов\Выгрузка Еком\\'

In [3]:
data_IM = pd.read_excel(rf'{way}\{month}.{year}_продажи еком.xlsx')

In [4]:
# смотрим продажи с последнего мониторинга
data = dt.datetime.isoweekday(dt.date.today())
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
data_IM = data_IM[data_IM['created'] >= f'{last_monitoring}']

In [5]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
1486,b3c29d16-b598-4f6c-b963-db4fd9344d06,2024-06-03 10:31:51.860,Золотое гладкое обручальное кольцо 2 мм,done,79127178709.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ0871315,4-06-033,1.09,1.09,NaN,585.00,ИФ ОБРУЧ ГЛАДКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 02-00,2406031031722,6870.00,MOBILE,2024-06-03 16:29:41.673,NaN,4978904,3442.75,2078609540456,4-06-033,0,0.00,0.00,6870.00
418,30d0abe1-4acb-4cc3-ae6a-e4d575c7cc2e,2024-06-03 10:36:46.750,Золотое кольцо с гранатом и бриллиантами с род...,done,79219427493.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1307184,1-01-267,1.08,0.99,NaN,585.00,ДК ГЕОМЕТРИЯ,КОЛЬЦО,КРАСНЫЙ,ДК КОЛЬЦА,NaN,20-02-1200-80706,2406031036886,9990.00,WEB,2024-06-03 11:21:23.823,NaN,1884978,4099.60,2078607845450,1-01-267,0,0.00,0.00,9990.00
1666,c845fe80-a9ef-4d3d-b5ad-3869c277b348,2024-06-03 07:30:19.600,Золотая подвеска Пяточка с фианитом,done,79205083842.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1001209,2-13-013,1.27,1.24,NaN,585.00,ИФ ДЕТСКИЕ,ПОДВЕС ДЕКОРАТИВНЫЙ,КРАСНЫЙ,ИФ ПОДВЕС ДЕКОР,SOKOLOV,34831,2406030730569,15080.00,MOBILE,2024-06-04 06:38:44.442,NaN,2823324,4445.88,2078608366354,2-13-013,0,0.00,0.00,15080.00
216,187202f2-eb72-45ab-889d-3bb2c1d2f32b,2024-06-03 04:42:25.586,Серебряное кольцо с фианитами и родированием,done,79016090029.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,YANDEX_PAY,Яндекс пэй для сайта,ТОВ1315651,3-05-090,1.26,1.13,NaN,925.00,СИ ПРОЧЕЕ,КОЛЬЦО,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ КОЛЬЦО ИФ,NaN,1100780-00775,2406030442320,790.00,WEB,2024-06-03 09:53:36.329,NaN,1696781,329.64,2078610668712,3-05-090,0,0.00,0.00,790.00
448,352d5dff-7156-4968-b489-615ea3688b1f,2024-06-03 18:00:59.271,Серебряное кольцо с фианитом и родированием,done,79770310356.00,Sidelnikova2704@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1055832,3-03-050,1.36,1.19,NaN,925.00,СИ ПОМОЛВ,КОЛЬЦО,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ КОЛЬЦО ИФ,NaN,121-1601#РОД,2406031800346,1240.00,MOBILE,2024-06-04 06:24:53.642,NaN,4858624,357.32,2078606753670,3-03-050,248,1.00,248.00,992.00


In [6]:
data_IM = data_IM.drop_duplicates()
data_IM = data_IM.reset_index(drop=True)

data_IM = data_IM[(data_IM['state_order'] != 'returned') & (data_IM['state_order'] != 'canceled')]
data_IM = data_IM[~((data_IM['delivery_type'] == 'PICKUP_NOW') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))|
                ((data_IM['delivery_type'] == 'PICKUP') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))]

In [7]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
176,3edc75f9-60de-4999-8bd3-198f7ddc723a,2024-06-03 08:54:55.562,Серебряная печатка с эмалью и родированием,done,79219139432.00,deu7878@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - APP,ТОВ1185201,1-01-239,3.65,3.61,NaN,925.00,СИ ЭМАЛЬ,КОЛЬЦО ПЕЧАТКА,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ КОЛЬЦО ПЕЧАТ,NaN,21-79-0000-28729,2406030854505,2860.00,MOBILE,2024-06-04 12:12:28.234,NaN,1545002,895.84,2078609266566,1-01-239,572,1.00,572.00,2288.00
406,8e0beba1-4793-4ff6-8e80-68a0b96a685d,2024-06-04 04:10:12.393,Золотые серьги,done,79833138251.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ1250491,3-05-011,0.43,0.43,NaN,375.00,ИФ КОНГО АЛМАЗН,СЕРЬГИ-КОНГО,КРАСНЫЙ,БК СЕРЬГИ,NaN,12.04_10#375,2406040410130,2763.60,MOBILE,2024-06-04 12:12:09.018,NaN,1145710,965.67,2078609290544,3-05-011,0,0.00,0.00,2763.60
16,05fc0244-50c8-4473-a314-55faeb2ab3f9,2024-06-03 08:45:41.329,Кольцо-печатка из серебра с эмалью,done,79276987264.00,u-13.13@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1219497,5-02-081,4.24,4.24,NaN,925.00,СИ ЭМАЛЬ,КОЛЬЦО ПЕЧАТКА,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ КОЛЬЦО ПЕЧАТ,NaN,061286,2406030845989,2500.00,WEB,2024-06-04 09:03:07.502,NaN,4141239,959.70,2078576453474,5-02-081,500,1.00,500.00,2000.00
79,1ebbd9c3-59f4-47fb-b749-b46bed1dba98,2024-06-03 14:36:11.903,Золотые серьги с кварцем и фианитами с родиров...,done,79770325074.00,BAYRAMOVA-1993@INTERNET.RU,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1084194,3-03-037,2.69,2.21,NaN,585.00,ДК С ОДНИМ КАМНЕМ,СЕРЬГИ,КРАСНЫЙ,ПДК СЕРЬГИ,NaN,ССК 856#КВЗГТ,2406031436492,13700.00,MOBILE,2024-06-04 08:44:10.717,NaN,5961597,8282.39,2078609231813,3-03-037,0,0.00,0.00,13700.00
698,f9053ce1-6569-4fc5-bc70-fa10100ac27d,2024-06-03 18:24:31.260,Золотое плоское обручальное кольцо 5 мм,done,79192693293.00,sere5imonov@yandex.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф рассрочка 18 мес,ТОВ0887596,2-10-028,3.77,3.77,NaN,585.00,ИФ ОБРУЧ ПЛОСКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 15-00,2406031824259,25630.00,MOBILE,2024-06-04 14:33:47.619,NaN,2004212,11566.88,2078608393845,2-10-028,0,0.00,0.00,25630.00


In [8]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format = '%Y-%m-%d')

In [9]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

data_IM['price'] = data_IM['price'].apply(str_float)
data_IM['price'] = data_IM['price'].astype(float)

data_IM['avg_net_weight'] = data_IM['avg_net_weight'].apply(str_float)
data_IM['avg_net_weight'] = data_IM['avg_net_weight'].astype(float)

In [10]:
data_IM = data_IM[data_IM['price'] > 1].reset_index(drop=True)
data_IM = data_IM[data_IM['avg_net_weight'] > 0].reset_index(drop=True)

In [11]:
data_IM['Товарное направление'] = np.where(
    data_IM['tg'].isin(['БК ПОДВЕС КУЛЬТ', 'БК СЕРЬГИ', 'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПОДВЕС ДЕКОР',  'БК ПЕЧАТКИ','БК АКСЕССУАРЫ','ОПТ БК']), 'БК',
    np.where(
        data_IM['tg'].isin(['ПДК СЕРЬГИ', 'ПДК КОЛЬЦА','ПДК ПОДВЕСКИ']), 'ПДК',
        np.where(
            data_IM['tg'].isin(['ДК СЕРЬГИ',  'ДК КОЛЬЦА',    'ДК ПОДВЕСКИ',    'ДК КОЛЬЦА ОБРУЧ']), 'ДК',
            np.where(
                data_IM['tg'].isin(['ИФ СЕРЬГИ',  'ИФ КОЛЬЦА', 'ИФ ПОДВЕС ДЕКОР', 'ИФ ПЕЧАТКИ', 'ИФ ПОДВЕС КУЛЬТ','ИФ КОЛЬЦА ОБРУЧ','ИФ АКСЕССУАРЫ']), 'ИФ',
                np.where(
                    data_IM['tg'].isin(['ЦБ ЦЕПИ', 'ЦБ БРАСЛЕТЫ']), 'ЦБ',
                    np.where(
                        data_IM['tg'].isin(['СИ ПОДВЕС БК','СИ КОЛЬЦО ИФ','СИ БРАСЛЕТ БК','СИ КОЛЬЦО БК','СИ ЦЕПЬ','СИ СЕРЬГИ БК','СИ СЕРЬГИ ИФ','СИ ПОДВЕС ИФ',
                                            'СИ СЕРЬГИ РАЗН','СИ СЕРЬГИ ЦВ/К',\
                                            'СИ КОЛЬЦО ЦВ/К','','СИ ШАРМЫ','СИ ОБРУЧАЛЬНОЕ','СИ КОЛЬЦО ПЕЧАТ','СИ ПОДВЕСЦВ/К','СИ БРАСЛЕТ СК','СИ ЖЕМЧУГ',\
                                            'СИ АКСЕССУАР','СИ ДФРС ПРОЕКТ','СИ КЕРАМИКА','СИ ПОСУДА','СИ ЯНТАРЬ','СИ ЭМАЛЬ','СИ ВЫРИЦА']), 'СИ',
                        np.where(data_IM['tg'].isin(['ЧАСЫ']), 'ЧАСЫ', 'другое'
                        )
                    )
                )
            )
        )
    )
)

data_IM['Товарное направление'].value_counts()

Товарное направление
СИ        144
БК        135
ИФ         94
ЦБ         70
ДК         64
ПДК        62
другое     21
Name: count, dtype: int64

In [12]:
proverka = data_IM[data_IM['Товарное направление']=='другое']
proverka['tg'].unique()

array(['ВЫРИЦА СЕРЕБРО', 'СИ КОЛЬЕ ЦВ/К', 'СИ СЕРЬГИ\xa0 ЦВ/К',
       'СИ КОЛЬЕ СК', 'ОПТ ИФ', 'ДК ПЕЧАТКИ', 'СИ ПОДВЕС\xa0ЦВ/К',
       'СИ БРАСЛЕТ БК Декор'], dtype=object)

In [13]:
data_IM = data_IM[~(data_IM['name'].str.contains('каучук')
        |data_IM['name'].str.contains('шнурок')
        |data_IM['name'].str.contains('текстиль')
        |data_IM['name'].str.contains('хлопок')
        |data_IM['name'].str.contains('шелк')
        |data_IM['name'].str.contains('на нитке')
        |data_IM['name'].str.contains('кожа')
        |data_IM['name'].str.contains('жемчуг'))]

In [14]:
data_IM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 1 to 589
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         574 non-null    object        
 1   created                    574 non-null    datetime64[ns]
 2   name                       574 non-null    object        
 3   state_order                574 non-null    object        
 4   recipient_phone            574 non-null    float64       
 5   recipient_email            185 non-null    object        
 6   state_item                 574 non-null    object        
 7   delivery_type              574 non-null    object        
 8   payment_type               574 non-null    object        
 9   employee_id                26 non-null     object        
 10  type_ibox                  574 non-null    object        
 11  type_name                  574 non-null    object        
 12  tov          

In [15]:
data_IM = data_IM[(data_IM['Товарное направление'].isin(['БК', 'ЦБ']))&((data_IM['proba'] == '585')|(data_IM['proba'] == 585.00))]

In [16]:
data_IM['Товарное направление'].value_counts()

Товарное направление
БК    95
ЦБ    51
Name: count, dtype: int64

In [17]:
data_IM['proba'].unique()

array([585.])

In [18]:
data_IM['date_status_order_done'].value_counts()

date_status_order_done
2024-06-04 17:30:56.771    1
2024-06-03 15:42:43.844    1
2024-06-04 09:04:23.879    1
2024-06-03 06:36:08.018    1
2024-06-04 10:32:12.149    1
                          ..
2024-06-03 09:59:35.424    1
2024-06-04 11:22:24.811    1
2024-06-04 11:22:24.872    1
2024-06-03 14:16:54.237    1
2024-06-04 06:24:03.057    1
Name: count, Length: 146, dtype: int64

In [19]:
previous_date = data_IM['date_status_order_done'].min()

In [20]:
filtr_date = data_IM['date_status_order_done'].max()
print(filtr_date)

2024-06-04 17:54:46.318000


In [21]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format='%Y-%m-%d')

In [22]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [23]:
data_today = data_IM.query("date_status_order_done >= @previous_date")
data_today.head()

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana,Товарное направление
2,005d3b2b-956b-47b9-bd25-6add5e5c3ba0,2024-06-03 18:13:02.163,Золотой крест TALANT,done,79185909408.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1172634,6-01-139,0.75,0.75,NaN,585.00,ИФ КРЕСТ,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ,БК ПОДВЕС КУЛЬТ,NaN,51-02-0000-28636,2406031813598,7293.00,MOBILE,2024-06-04,NaN,8352349,2518.50,2078609166434,6-01-139,1458,1.00,1458.00,5835.00,БК
7,01970073-614d-4574-9753-fc81184bcad6,2024-06-04 06:58:41.316,Золотой пустотелый браслет Нонна,done,79514772889.00,gfl@mail.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ1186473,4-01-163,0.79,0.79,NaN,585.00,НОННА ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,"НБ12-200ПГ D0,4",2406040658970,5970.00,MOBILE,2024-06-04,NaN,1241418,2690.73,2078610864351,4-01-163,0,0.00,0.00,5970.00,ЦБ
18,0773145e-3707-4c13-ba03-31cbb2bf03d0,2024-06-04 13:47:40.784,Золотой пустотелый браслет Тройной ромб с алма...,done,79870263843.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ01262174,4-06-024,1.22,1.22,NaN,585.00,РОМБ ТРОЙНОЙ ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,41-01-0040-30204,2406041347493,12502.00,MOBILE,2024-06-04,NaN,1180825,3894.39,2078607113209,4-06-024,5174,0.48,2500.21,10001.79,ЦБ
33,10c56660-0e79-4572-aa9d-601ef34458c6,2024-06-04 10:17:14.288,Золотой пустотелый браслет Тройной ромб с алма...,done,79963132394.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ01262172,3-05-113,1.17,1.17,NaN,585.00,РОМБ ТРОЙНОЙ ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,NaN,41-01-0040-30204,2406041017478,10907.00,MOBILE,2024-06-04,NaN,1180825,4359.12,2078608240985,3-05-113,2181,1.00,2181.00,8726.00,ЦБ
51,1be13c2c-fb0f-455d-bb17-9fe72e48da08,2024-06-03 13:37:35.667,Золотое гладкое обручальное кольцо 4 мм,done,79505378802.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ0832972,4-05-022,2.70,2.70,NaN,585.00,ИФ ОБРУЧ ГЛАДКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 04-00#ТД,2406031337603,12890.00,MOBILE,2024-06-04,NaN,2002944,9253.42,2078606629733,4-05-022,0,0.00,0.00,12890.00,БК


In [24]:
data_today['date_status_order_done'].unique()

<DatetimeArray>
['2024-06-04 00:00:00']
Length: 1, dtype: datetime64[ns]

In [25]:
data_today.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 2 to 580
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         88 non-null     object        
 1   created                    88 non-null     datetime64[ns]
 2   name                       88 non-null     object        
 3   state_order                88 non-null     object        
 4   recipient_phone            88 non-null     float64       
 5   recipient_email            31 non-null     object        
 6   state_item                 88 non-null     object        
 7   delivery_type              88 non-null     object        
 8   payment_type               88 non-null     object        
 9   employee_id                8 non-null      object        
 10  type_ibox                  88 non-null     object        
 11  type_name                  88 non-null     object        
 12  tov           

In [26]:
data_today['цена за гр'] = data_today['price_wo_manzana'] / data_today['avg_weight']
data_today['цена за гр'].describe()

count      88.00
mean     7168.89
std      1535.90
min      4481.38
25%      6273.14
50%      7194.08
75%      7853.84
max     12851.85
Name: цена за гр, dtype: float64

In [27]:
data_today['avg_weight'].describe()

count   88.00
mean     2.32
std      1.78
min      0.20
25%      0.94
50%      2.21
75%      3.06
max     12.48
Name: avg_weight, dtype: float64

In [28]:
data_today = data_today[~data_today['tg'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО')]

In [29]:
data_today = data_today[~data_today['client_type'].str.contains('OZON')]

In [30]:
print('Средняя цена продажи еком: ', (data_today['price_wo_manzana'].sum()/data_today['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6944.0


In [31]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')

In [32]:
data_today.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\online_sales_{date_now}.xlsx', index=False)

In [33]:
tn = data_today.groupby('tg', as_index=False)[['price_wo_manzana', 'avg_weight', 'cost_price']].agg('sum')
tn['price_gramm'] = (tn['price_wo_manzana'] / tn['avg_weight']).round(0)

In [34]:
tn['price_gramm'] = tn['price_gramm'].astype('int')

In [35]:
tn['Наценка продажи онлайн'] =  tn['price_wo_manzana'] / tn['cost_price'] - 1

In [37]:
print('Средняя цена продажи еком: ', (tn['price_wo_manzana'].sum() / tn['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6944.0


In [38]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\ecom_sales_tg\\ecom_sales_{date_now}.xlsx', index=False)

In [39]:
tn = tn.drop(columns=['price_wo_manzana', 'avg_weight', 'cost_price', 'Наценка продажи онлайн'])

In [40]:
tn.columns = ['tg', '585*Золотой онлайн продажи']

In [41]:
day

5

In [42]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)